# DATASETS & DATALOADERS

Code for processing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled fom model training code for better readability and modularity py torch provides tow data primitive: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` that allow you to use pre-loaded datasets as well as your own data. `Dataset` stores the samples of their corresponding labels and `DataLoader` wraps an iterable around the dataset to enable easy access tot the samples.

PyTorch domain libraries (TorchVsion, TorchAudio, TorchText) provide a number of pre-loaded datasets that subclass torch.utils.data.Dataset and implement functions specific for the particular data. They can be used to prototype and benchmark your model.

## Loading a dataset

Loading a sample dataset (FashionMNIST). 

In [2]:
# same as in quickstart
# importing torch

import torch # importing main torch library
from torch import nn # importing torch neural network related submodule
from torch.utils.data import DataLoader # importing DataLoader class
from torchvision import datasets # importing datasets class from vision-specific library torchvision
# other domain specific libraries are torchtext and torchaudio
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

# Downloading MNIST datasets
training_data = datasets.FashionMNIST(
    root =  "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

# Iterating and Visualizing the dataset

In [ ]:
labels_map = {
     0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}